# MIP における論理演算の扱い

$a, b$ を 0-1 決定変数とする. 
このとき以下のような制約を持つ決定変数 $c$ の定め方を説明する.

- AND: $c = a \land b$
- OR: $c = a \lor b$
- NAND: $c = a \uparrow b$
- NOR: $c = a \downarrow b$
- XOR: $c = a \veebar b$
- XNOR: $c = (a \Leftrightarrow b)$

## 定式化と実装

In [1]:
from ortools.sat.python import cp_model

In [2]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print(f"{v}={self.value(v)}", end=" ")
        print()

    @property
    def solution_count(self):
        return self.__solution_count

### AND

$a \land b \Rightarrow c$ と $c \Rightarrow a \land b$ を表現すればよい. 
($x \Rightarrow y$ は $x \leq y$ で書けることに注意)

- $a + b \leq 1 + c$
- $2 c \leq a + b$

In [3]:
model = cp_model.CpModel()

a = model.new_bool_var("A")
b = model.new_bool_var("B")
c = model.new_bool_var(f"{a}and{b}")

model.add(a + b <= 1 + c)
model.add(2 * c <= a + b)

solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([a, b, c])
solver.parameters.enumerate_all_solutions = True
status = solver.solve(model, solution_printer)

A=0 B=0 AandB=0 
A=0 B=1 AandB=0 
A=1 B=1 AandB=1 
A=1 B=0 AandB=0 


### OR

$a \lor b \Rightarrow c$ と $c \Rightarrow a \lor b$ を表現すればよい. 

- $a + b \leq 2 c$
- $c \leq a + b$

In [4]:
model = cp_model.CpModel()

a = model.new_bool_var("A")
b = model.new_bool_var("B")
c = model.new_bool_var(f"{a}or{b}")

model.add(a + b <= 2 * c)
model.add(c <= a + b)

solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([a, b, c])
solver.parameters.enumerate_all_solutions = True
status = solver.solve(model, solution_printer)

A=0 B=0 AorB=0 
A=0 B=1 AorB=1 
A=1 B=1 AorB=1 
A=1 B=0 AorB=1 


### NAND

AND の定式化における $c$ を $1 - c$ に置き換えればよい. 

- $a + b \leq 1 + (1 - c)$
- $2 (1 - c) \leq a + b$

あるいは AND で定式化した $c$ に対して $1 - c$ を用いる. 

In [5]:
model = cp_model.CpModel()

a = model.new_bool_var("A")
b = model.new_bool_var("B")
c = model.new_bool_var(f"{a}nand{b}")

model.add(a + b <= 1 + (1 - c))
model.add(2 * (1 - c) <= a + b)

solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([a, b, c])
solver.parameters.enumerate_all_solutions = True
status = solver.solve(model, solution_printer)

A=1 B=1 AnandB=0 
A=1 B=0 AnandB=1 
A=0 B=0 AnandB=1 
A=0 B=1 AnandB=1 


### NOR

OR の定式化における $c$ を $1 - c$ に置き換えれば良い. 

- $a + b \leq 2 (1 - c)$
- $1 - c \leq a + b$

あるいは OR で定式化した $c$ に対して $1 - c$ を用いる. 

In [6]:
model = cp_model.CpModel()

a = model.new_bool_var("A")
b = model.new_bool_var("B")
c = model.new_bool_var(f"{a}nor{b}")

model.add(a + b <= 2 * (1 - c))
model.add(1 - c <= a + b)

solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([a, b, c])
solver.parameters.enumerate_all_solutions = True
status = solver.solve(model, solution_printer)

A=0 B=1 AnorB=0 
A=0 B=0 AnorB=1 
A=1 B=0 AnorB=0 
A=1 B=1 AnorB=0 


### XOR

$(a = 0 \land b = 0) \Rightarrow c = 0$, $(a = 1 \land b = 1) \Rightarrow c = 0$, $(a = 1 \land b = 0) \Rightarrow c = 1$, $(a = 0 \land b = 1) \Rightarrow c = 1$ を全て表現する. 

- $(1 - a) + (1 - b) \leq 1 + (1 - c)$
- $a + b \leq 1 + (1 - c)$
- $a + (1 - b) \leq 1 + c$
- $(1 - a) + b \leq 1 + c$

In [7]:
model = cp_model.CpModel()

a = model.new_bool_var("A")
b = model.new_bool_var("B")
c = model.new_bool_var(f"{a}xor{b}")

model.add((1 - a) + (1 - b) <= 1 + (1 - c))
model.add(a + b <= 1 + (1 - c))
model.add(a + (1 - b) <= 1 + c)
model.add((1 - a) + b <= 1 + c)

solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([a, b, c])
solver.parameters.enumerate_all_solutions = True
status = solver.solve(model, solution_printer)

A=0 B=0 AxorB=0 
A=0 B=1 AxorB=1 
A=1 B=1 AxorB=0 
A=1 B=0 AxorB=1 


### XNOR

XOR の定式化における $c$ を $1 - c$ に置き換えれば良い. 

- $(1 - a) + (1 - b) \leq 1 + c$
- $a + b \leq 1 + c$
- $a + (1 - b) \leq 1 + (1 - c)$
- $(1 - a) + b \leq 1 + (1 - c)$

あるいは XOR で定式化した $c$ に対して $1 - c$ を用いる. 

In [8]:
model = cp_model.CpModel()

a = model.new_bool_var("A")
b = model.new_bool_var("B")
c = model.new_bool_var(f"{a}xor{b}")

model.add((1 - a) + (1 - b) <= 1 + c)
model.add(a + b <= 1 + c)
model.add(a + (1 - b) <= 1 + (1 - c))
model.add((1 - a) + b <= 1 + (1 - c))

solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([a, b, c])
solver.parameters.enumerate_all_solutions = True
status = solver.solve(model, solution_printer)

A=0 B=1 AxorB=0 
A=0 B=0 AxorB=1 
A=1 B=0 AxorB=0 
A=1 B=1 AxorB=1 
